In [ ]:
train_path = "your train_data path"
model_path = "your model path"

In [7]:
import glob
import imageio.v2 as imageio
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import tqdm
import albumentations as A
import csv
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2

def get_label(f):
    # 根據檔案名稱獲取標籤
    if 'incendio' in f.lower():
        label = 1
    elif 'aqua' in f.lower():
        label = 2
    elif 'arresto' in f.lower():
        label = 3
    elif 'alohomora' in f.lower():
        label = 4
    elif 'lumos' in f.lower():
        label = 5
    elif 'null' in f.lower():
        label = 0
    return label

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 6)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        #x = self.fc1(x.view(x.size(0), -1))
        x = self.relu3(x)
        x = self.fc2(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ffs = glob.glob(f'{train_path}/*/*.png')
train_list = []
for f in tqdm.tqdm(ffs):
    im = imageio.imread(f)

    #print(im.shape, f)
    label = get_label(f)
    train_list.append([im, label])




100%|████████████████████████████████████████████████████████████████████████████| 1253/1253 [00:00<00:00, 4082.29it/s]


In [10]:
def run_epoch(data, model, criterion, optimizer, device, is_train=True):
    total_loss = 0
    count = 0

    for dd in tqdm.tqdm(data):
        im, label = dd

        if is_train:
            transform = A.Compose([
              # A.Resize(28, 28),
              A.ShiftScaleRotate(p=0.5),
              A.OpticalDistortion(p=0.5),
              A.GridDistortion(p=0.5),
          ])
            im = transform(image=im)['image']

        im_d = torch.from_numpy(im[None, ...][None, ...]).to(device).float() / 255
        label_d = torch.from_numpy(np.array([label])).to(device)

        output_d = model(im_d)
        loss = criterion(output_d, label_d.long())

        if is_train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        count += 1

    return total_loss / count if count > 0 else 0


In [11]:
import random
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import numpy as np
import albumentations as A

# Assuming CNN is defined somewhere
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

epochs = 1000
csv_filename = 'training_results_null.csv'
header = ['Epoch', 'Training Loss', 'Validation Loss']

# Splitting the dataset into training and validation
validation_split = 0.05
test_spilit = 0.2
split_idx = int(len(train_list) * (1 - validation_split))
random.shuffle(train_list)
train_data, validation_data = train_list[:split_idx], train_list[split_idx:]

# Writing the header to CSV
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

for epoch in range(epochs):
    model.train()
    random.shuffle(train_data)
    train_loss = run_epoch(train_data, model, criterion, optimizer, device, is_train=True)

    model.eval()
    with torch.no_grad():
        validation_loss = run_epoch(validation_data, model, criterion, optimizer, device, is_train=False)

    print(f"Epoch {epoch+1}, Training Loss: {train_loss:.5f}, Validation Loss: {validation_loss:.5f}")

    # Save the results to CSV
    with open(csv_filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch + 1, train_loss, validation_loss])

    # Save the model periodically
    if epoch % 100 == 0:
        torch.save(model, f'model_path/Null-add_{epoch}.pt')
        print("Saving the model as Nullv1.pt")




100%|████████████████████████████████████████████████████████████████████████████████| 63/63 [00:00<00:00, 3797.60it/s]


Epoch 1, Training Loss: 1.37086, Validation Loss: 1.30539
Saving the model as Nullv1.pt


100%|████████████████████████████████████████████████████████████████████████████████| 63/63 [00:00<00:00, 3802.85it/s]


Epoch 2, Training Loss: 1.33581, Validation Loss: 1.18058


100%|████████████████████████████████████████████████████████████████████████████████| 63/63 [00:00<00:00, 3912.54it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 63/63 [00:00<00:00, 3844.74it/s]


Epoch 38, Training Loss: 0.56216, Validation Loss: 0.30930


100%|████████████████████████████████████████████████████████████████████████████████| 63/63 [00:00<00:00, 2915.12it/s]


Epoch 39, Training Loss: 0.54218, Validation Loss: 0.26271


 16%|████████████▋                                                                 | 193/1190 [00:00<00:03, 314.09it/s]

KeyboardInterrupt

